In [23]:
# Install necessary packages

In [24]:
# pip install open-fdd --upgrade

In [25]:
import pandas as pd
import numpy as np


In [ ]:
# Load your data
ahu_data = r"C:\Users\bbartling\Documents\Midtown_PD_Master.csv"
df = pd.read_csv(ahu_data)

df.head()

In [ ]:
# Check for NaN values in each column
for col in df.columns:
    if df[col].isnull().any():
        print(f"NaN values found in column: {col}")

        # Remove rows with any NaN values, then forward and backfill
        df = df.dropna().ffill().bfill()
        print("DataFrame has been cleaned for NaNs and has also been forward and backfilled.")

In [ ]:
# Filter the DataFrame
filtered_df = df[(df['SA_FanSpeed'] > 15.0) & (df['Oa_Temp'] < 10.0)]

# Get the description of the 'OA_Damper' column
oa_damper_description = filtered_df['OA_RA_Damper'].describe()

# Print the description
oa_damper_description

In [ ]:
# Checking data types of each column
print("Data Types:")
print(df.dtypes)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print("\nMax Values:")
    print(df.max(numeric_only=True))

In [ ]:
# OPTIONAL CONVERT AO's based on above info

'''

# the floats between 0.0 and 100.0 so we need to convert to 0.0 and 1.0 ranges
percentage_columns = [
    ["SA_FanVFD"],
    ["OA_RA_Damper"],
    ["SA_FanSpeed"],
]

for col in percentage_columns:
    df[col] = df[col] / 100.0

df.head()

'''

In [ ]:
# Convert the timestamp column to datetime and set it as the index
df["timestamp"] = pd.to_datetime(df["timestamp"])
df.set_index("timestamp", inplace=True)

df.head()

In [ ]:
# See fault code 4 section down below

df_copy_for_fc4 = df.copy()

In [ ]:


# Configuration dictionary
config_dict = {
    # used for report name
    'AHU_NAME': "MZVAV_1",

    # timestamp column name
    'INDEX_COL_NAME': "timestamp",

    'DUCT_STATIC_COL': "SaStatic",
    'DUCT_STATIC_SETPOINT_COL': "Static_SP",
    'SUPPLY_VFD_SPEED_COL': "SA_FanSpeed",

    'MAT_COL': "MA_Temp",
    'OAT_COL': "OaTemp",
    'SAT_COL': "HC1_DaTemp",
    'RAT_COL': "RA_Temp",

    'HEATING_SIG_COL': None,  
    'COOLING_SIG_COL': "cooling_signal",  # calculated in pandas for this AHU
    'ECONOMIZER_SIG_COL': "OA_RA_Damper",

    # Set to None to potentially skip Fault Condition 6
    'SUPPLY_FAN_AIR_VOLUME_COL': "SA_Flow_CFM",  

    # Set to None to potentially skip Fault Condition 14
    'CLG_COIL_ENTER_TEMP_COL': None,
    'CLG_COIL_LEAVE_TEMP_COL': None,

    # Set to None to potentially skip Fault Condition 15
    'HTG_COIL_ENTER_TEMP_COL': None,
    'HTG_COIL_LEAVE_TEMP_COL': None,

    'SAT_SETPOINT_COL': "Eff_DaTempSP",
    'CONSTANT_LEAVE_TEMP_SP': False,
    'CONSTANT_LEAVE_TEMP_SP_VAL': 55.0,

    'VFD_SPEED_PERCENT_ERR_THRES': 0.05,
    'VFD_SPEED_PERCENT_MAX': 0.99,
    'DUCT_STATIC_INCHES_ERR_THRES': 0.1,
    'OUTDOOR_DEGF_ERR_THRES': 5.0,
    'MIX_DEGF_ERR_THRES': 5.0,
    'RETURN_DEGF_ERR_THRES': 2.0,
    'SUPPLY_DEGF_ERR_THRES': 2.0,
    'COIL_TEMP_ENTER_ERR_THRES': 2.0,
    'COIL_TEMP_LEAV_ERR_THRES': 2.0,
    'DELTA_T_SUPPLY_FAN': 2.0,

    'DELTA_OS_MAX': 3,
    'AHU_MIN_OA_DPR': 0.20, # Found from the previous summary stats
    'OAT_RAT_DELTA_MIN': 10, # Intentional as type int to show new error handling in fc6
    'AIRFLOW_ERR_THRES': 0.3,
    'AHU_MIN_OA_CFM_DESIGN': 2500,
    'TROUBLESHOOT_MODE': False,
    'ROLLING_WINDOW_SIZE': 10
}

In [ ]:
# create empty dict to hold fault counts which we can do something with later

fault_counts = {}

In [ ]:
from open_fdd.air_handling_unit.faults import FaultConditionOne
from open_fdd.air_handling_unit.reports import FaultCodeOneReport

# Create an instance of FaultConditionOne
fc1 = FaultConditionOne(config_dict)
fc1_required_columns = fc1.get_required_columns()
print(fc1_required_columns)

# Apply the fault condition to the DataFrame
df_fc1 = fc1.apply(df)
fault_counts["fc1_fault_sum"] = df_fc1["fc1_flag"].sum()
print(f"FC1 Fault Sum: {fault_counts['fc1_fault_sum']}")

# Retrieve and print all the metrics (keys and values) from summarize_fault_times
fc1_report = FaultCodeOneReport(config_dict)
summary_metrics = fc1_report.summarize_fault_times(df_fc1)

# Print each key and value
for key, value in summary_metrics.items():
    print(f"{key}: {value}")

# Generate and display the report if there is a fault
if fault_counts['fc1_fault_sum'] != 0:
    fc1_report.display_report_in_ipython(df_fc1)
else:
    print("No faults found.")


In [ ]:
from open_fdd.air_handling_unit.faults import FaultConditionTwo
from open_fdd.air_handling_unit.reports import FaultCodeTwoReport

# Create an instance of FaultConditionTwo
fc2 = FaultConditionTwo(config_dict)

# Retrieve and print the required columns for this fault condition
fc2_required_columns = fc2.get_required_columns()
print(fc2_required_columns)

# Apply the fault condition to the DataFrame
df_fc2 = fc2.apply(df)

# Calculate and store the fault sum in fault_counts
fault_counts["fc2_fault_sum"] = df_fc2["fc2_flag"].sum()

# Print the fault sum
print(f"FC2 Fault Sum: {fault_counts['fc2_fault_sum']}")

# Retrieve and print all the metrics (keys and values) from summarize_fault_times
fc2_report = FaultCodeTwoReport(config_dict)
summary_metrics = fc2_report.summarize_fault_times(df_fc2)

# Print each key and value
for key, value in summary_metrics.items():
    print(f"{key}: {value}")

# Generate and display the report if there is a fault
if fault_counts['fc2_fault_sum'] != 0:
    fc2_report.display_report_in_ipython(df_fc2)
else:
    print("No faults found.")


In [ ]:
from open_fdd.air_handling_unit.faults import FaultConditionThree
from open_fdd.air_handling_unit.reports import FaultCodeThreeReport

# Create an instance of FaultConditionThree
fc3 = FaultConditionThree(config_dict)

# Retrieve and print the required columns for this fault condition
fc3_required_columns = fc3.get_required_columns()
print(fc3_required_columns)

# Apply the fault condition to the DataFrame
df_fc3 = fc3.apply(df)

# Calculate and store the fault sum in fault_counts
fault_counts["fc3_fault_sum"] = df_fc3["fc3_flag"].sum()

# Print the fault sum
print(f"FC3 Fault Sum: {fault_counts['fc3_fault_sum']}")

# Retrieve and print all the metrics (keys and values) from summarize_fault_times
fc3_report = FaultCodeThreeReport(config_dict)
summary_metrics = fc3_report.summarize_fault_times(df_fc3)

# Print each key and value
for key, value in summary_metrics.items():
    print(f"{key}: {value}")

# Generate and display the report if there is a fault
if fault_counts['fc3_fault_sum'] != 0:
    fc3_report.display_report_in_ipython(df_fc3)
else:
    print("No faults found.")


In [ ]:
from open_fdd.air_handling_unit.faults import FaultConditionFour
from open_fdd.air_handling_unit.reports import FaultCodeFourReport

'''
DONT forget fault 4 is resampled data to hourly sums to
flag hunting in control system so this df needs to be
treated seperately and cannot combine back into the orignal
df because of the resampling process hence the df_copy_for_fc4
'''

# Create an instance of FaultConditionFour
fc4 = FaultConditionFour(config_dict)

# Retrieve and print the required columns for this fault condition
fc4_required_columns = fc4.get_required_columns()
print(fc4_required_columns)

# Apply the fault condition to the DataFrame
df_fc4 = fc4.apply(df_copy_for_fc4)

# Calculate and store the fault sum in fault_counts
fault_counts["fc4_fault_sum"] = df_fc4["fc4_flag"].sum()

# Print the fault sum
print(f"FC4 Fault Sum: {fault_counts['fc4_fault_sum']}")

# Retrieve and print all the metrics (keys and values) from summarize_fault_times
fc4_report = FaultCodeFourReport(config_dict)
summary_metrics = fc4_report.summarize_fault_times(df_fc4)

# Print each key and value
for key, value in summary_metrics.items():
    print(f"{key}: {value}")

# Generate and display the report if there is a fault
if fault_counts['fc4_fault_sum'] != 0:
    fc4_report.display_report_in_ipython(df_fc4)
else:
    print("No faults found.")


In [ ]:
from open_fdd.air_handling_unit.faults import FaultConditionFive
from open_fdd.air_handling_unit.reports import FaultCodeFiveReport

# Create an instance of FaultConditionFive
fc5 = FaultConditionFive(config_dict)

# Retrieve and print the required columns for this fault condition
fc5_required_columns = fc5.get_required_columns()
print(fc5_required_columns)

# Apply the fault condition to the DataFrame
df_fc5 = fc5.apply(df)

# Calculate and store the fault sum in fault_counts
fault_counts["fc5_fault_sum"] = df_fc5["fc5_flag"].sum()

# Print the fault sum
print(f"FC5 Fault Sum: {fault_counts['fc5_fault_sum']}")

# Retrieve and print all the metrics (keys and values) from summarize_fault_times
fc5_report = FaultCodeFiveReport(config_dict)
summary_metrics = fc5_report.summarize_fault_times(df_fc5)

# Print each key and value
for key, value in summary_metrics.items():
    print(f"{key}: {value}")

# Generate and display the report if there is a fault
if fault_counts['fc5_fault_sum'] != 0:
    fc5_report.display_report_in_ipython(df_fc5)
else:
    print("No faults found.")


In [ ]:
from open_fdd.air_handling_unit.faults import FaultConditionSix
from open_fdd.air_handling_unit.reports import FaultCodeSixReport

# Create an instance of FaultConditionSix
fc6 = FaultConditionSix(config_dict)

# Retrieve and print the required columns for this fault condition
fc6_required_columns = fc6.get_required_columns()
print(fc6_required_columns)

# Apply the fault condition to the DataFrame
df_fc6 = fc6.apply(df)

# Calculate and store the fault sum in fault_counts
fault_counts["fc6_fault_sum"] = df_fc6["fc6_flag"].sum()

# Print the fault sum
print(f"FC6 Fault Sum: {fault_counts['fc6_fault_sum']}")

# Retrieve and print all the metrics (keys and values) from summarize_fault_times
fc6_report = FaultCodeSixReport(config_dict)
summary_metrics = fc6_report.summarize_fault_times(df_fc6)

# Print each key and value
for key, value in summary_metrics.items():
    print(f"{key}: {value}")

# Generate and display the report if there is a fault
if fault_counts['fc6_fault_sum'] != 0:
    fc6_report.display_report_in_ipython(df_fc6)
else:
    print("No faults found.")


In [ ]:
from open_fdd.air_handling_unit.faults import FaultConditionSeven
from open_fdd.air_handling_unit.reports import FaultCodeSevenReport

# Create an instance of FaultConditionSeven
fc7 = FaultConditionSeven(config_dict)

# Retrieve and print the required columns for this fault condition
fc7_required_columns = fc7.get_required_columns()
print(fc7_required_columns)

# Apply the fault condition to the DataFrame
df_fc7 = fc7.apply(df)

# Calculate and store the fault sum in fault_counts
fault_counts["fc7_fault_sum"] = df_fc7["fc7_flag"].sum()

# Print the fault sum
print(f"FC7 Fault Sum: {fault_counts['fc7_fault_sum']}")

# Retrieve and print all the metrics (keys and values) from summarize_fault_times
fc7_report = FaultCodeSevenReport(config_dict)
summary_metrics = fc7_report.summarize_fault_times(df_fc7)

# Print each key and value
for key, value in summary_metrics.items():
    print(f"{key}: {value}")

# Generate and display the report if there is a fault
if fault_counts['fc7_fault_sum'] != 0:
    fc7_report.display_report_in_ipython(df_fc7)
else:
    print("No faults found.")


In [ ]:
from open_fdd.air_handling_unit.faults import FaultConditionEight
from open_fdd.air_handling_unit.reports import FaultCodeEightReport

# Create an instance of FaultConditionEight
fc8 = FaultConditionEight(config_dict)

# Retrieve and print the required columns for this fault condition
fc8_required_columns = fc8.get_required_columns()
print(fc8_required_columns)

# Apply the fault condition to the DataFrame
df_fc8 = fc8.apply(df)

# Calculate and store the fault sum in fault_counts
fault_counts["fc8_fault_sum"] = df_fc8["fc8_flag"].sum()

# Print the fault sum
print(f"FC8 Fault Sum: {fault_counts['fc8_fault_sum']}")

# Retrieve and print all the metrics (keys and values) from summarize_fault_times
fc8_report = FaultCodeEightReport(config_dict)
summary_metrics = fc8_report.summarize_fault_times(df_fc8)

# Print each key and value
for key, value in summary_metrics.items():
    print(f"{key}: {value}")

# Generate and display the report if there is a fault
if fault_counts['fc8_fault_sum'] != 0:
    fc8_report.display_report_in_ipython(df_fc8)
else:
    print("No faults found.")


In [ ]:
from open_fdd.air_handling_unit.faults import FaultConditionNine
from open_fdd.air_handling_unit.reports import FaultCodeNineReport

# Create an instance of FaultConditionNine
fc9 = FaultConditionNine(config_dict)

# Retrieve and print the required columns for this fault condition
fc9_required_columns = fc9.get_required_columns()
print(fc9_required_columns)

# Apply the fault condition to the DataFrame
df_fc9 = fc9.apply(df)

# Calculate and store the fault sum in fault_counts
fault_counts["fc9_fault_sum"] = df_fc9["fc9_flag"].sum()

# Print the fault sum
print(f"FC9 Fault Sum: {fault_counts['fc9_fault_sum']}")

# Retrieve and print all the metrics (keys and values) from summarize_fault_times
fc9_report = FaultCodeNineReport(config_dict)
summary_metrics = fc9_report.summarize_fault_times(df_fc9)

# Print each key and value
for key, value in summary_metrics.items():
    print(f"{key}: {value}")

# Generate and display the report if there is a fault
if fault_counts['fc9_fault_sum'] != 0:
    fc9_report.display_report_in_ipython(df_fc9)
else:
    print("No faults found.")

In [ ]:
from open_fdd.air_handling_unit.faults import FaultConditionTen
from open_fdd.air_handling_unit.reports import FaultCodeTenReport

# Create an instance of FaultConditionTen
fc10 = FaultConditionTen(config_dict)

# Retrieve and print the required columns for this fault condition
fc10_required_columns = fc10.get_required_columns()
print(fc10_required_columns)

# Apply the fault condition to the DataFrame
df_fc10 = fc10.apply(df)

# Calculate and store the fault sum in fault_counts
fault_counts["fc10_fault_sum"] = df_fc10["fc10_flag"].sum()

# Print the fault sum
print(f"FC10 Fault Sum: {fault_counts['fc10_fault_sum']}")

# Retrieve and print all the metrics (keys and values) from summarize_fault_times
fc10_report = FaultCodeTenReport(config_dict)
summary_metrics = fc10_report.summarize_fault_times(df_fc10)

# Print each key and value
for key, value in summary_metrics.items():
    print(f"{key}: {value}")

# Generate and display the report if there is a fault
if fault_counts['fc10_fault_sum'] != 0:
    fc10_report.display_report_in_ipython(df_fc10)
else:
    print("No faults found.")

In [ ]:
from open_fdd.air_handling_unit.faults import FaultConditionEleven
from open_fdd.air_handling_unit.reports import FaultCodeElevenReport

# Create an instance of FaultConditionEleven
fc11 = FaultConditionEleven(config_dict)

# Retrieve and print the required columns for this fault condition
fc11_required_columns = fc11.get_required_columns()
print(fc11_required_columns)

# Apply the fault condition to the DataFrame
df_fc11 = fc11.apply(df)

# Calculate and store the fault sum in fault_counts
fault_counts["fc11_fault_sum"] = df_fc11["fc11_flag"].sum()

# Print the fault sum
print(f"FC11 Fault Sum: {fault_counts['fc11_fault_sum']}")

# Retrieve and print all the metrics (keys and values) from summarize_fault_times
fc11_report = FaultCodeElevenReport(config_dict)
summary_metrics = fc11_report.summarize_fault_times(df_fc11)

# Print each key and value
for key, value in summary_metrics.items():
    print(f"{key}: {value}")

# Generate and display the report if there is a fault
if fault_counts['fc11_fault_sum'] != 0:
    fc11_report.display_report_in_ipython(df_fc11)
else:
    print("No faults found.")

In [ ]:
from open_fdd.air_handling_unit.faults import FaultConditionTwelve
from open_fdd.air_handling_unit.reports import FaultCodeTwelveReport

# Create an instance of FaultConditionTwelve
fc12 = FaultConditionTwelve(config_dict)

# Retrieve and print the required columns for this fault condition
fc12_required_columns = fc12.get_required_columns()
print(fc12_required_columns)

# Apply the fault condition to the DataFrame
df_fc12 = fc12.apply(df)

# Calculate and store the fault sum in fault_counts
fault_counts["fc12_fault_sum"] = df_fc12["fc12_flag"].sum()

# Print the fault sum
print(f"FC12 Fault Sum: {fault_counts['fc12_fault_sum']}")

# Retrieve and print all the metrics (keys and values) from summarize_fault_times
fc12_report = FaultCodeTwelveReport(config_dict)
summary_metrics = fc12_report.summarize_fault_times(df_fc12)

# Print each key and value
for key, value in summary_metrics.items():
    print(f"{key}: {value}")

# Generate and display the report if there is a fault
if fault_counts['fc12_fault_sum'] != 0:
    fc12_report.display_report_in_ipython(df_fc12)
else:
    print("No faults found.")

In [ ]:
from open_fdd.air_handling_unit.faults import FaultConditionThirteen
from open_fdd.air_handling_unit.reports import FaultCodeThirteenReport

# Create an instance of FaultConditionThirteen
fc13 = FaultConditionThirteen(config_dict)

# Retrieve and print the required columns for this fault condition
fc13_required_columns = fc13.get_required_columns()
print(fc13_required_columns)

# Apply the fault condition to the DataFrame
df_fc13 = fc13.apply(df)

# Calculate and store the fault sum in fault_counts
fault_counts["fc13_fault_sum"] = df_fc13["fc13_flag"].sum()

# Print the fault sum
print(f"FC13 Fault Sum: {fault_counts['fc13_fault_sum']}")

# Retrieve and print all the metrics (keys and values) from summarize_fault_times
fc13_report = FaultCodeThirteenReport(config_dict)
summary_metrics = fc13_report.summarize_fault_times(df_fc13)

# Print each key and value
for key, value in summary_metrics.items():
    print(f"{key}: {value}")

# Generate and display the report if there is a fault
if fault_counts['fc13_fault_sum'] != 0:
    fc13_report.display_report_in_ipython(df_fc13)
else:
    print("No faults found.")

In [ ]:
from open_fdd.air_handling_unit.faults import FaultConditionFourteen
from open_fdd.air_handling_unit.reports import FaultCodeFourteenReport

# Create an instance of FaultConditionFourteen
fc14 = FaultConditionFourteen(config_dict)

# Retrieve and print the required columns for this fault condition
fc14_required_columns = fc14.get_required_columns()
print(fc14_required_columns)

# Apply the fault condition to the DataFrame
df_fc14 = fc14.apply(df)

# Calculate and store the fault sum in fault_counts
fault_counts["fc14_fault_sum"] = df_fc14["fc14_flag"].sum()

# Print the fault sum
print(f"FC14 Fault Sum: {fault_counts['fc14_fault_sum']}")

# Retrieve and print all the metrics (keys and values) from summarize_fault_times
fc14_report = FaultCodeFourteenReport(config_dict)
summary_metrics = fc14_report.summarize_fault_times(df_fc14)

# Print each key and value
for key, value in summary_metrics.items():
    print(f"{key}: {value}")

# Generate and display the report if there is a fault
if fault_counts['fc14_fault_sum'] != 0:
    fc14_report.display_report_in_ipython(df_fc14)
else:
    print("No faults found.")


In [ ]:
from open_fdd.air_handling_unit.faults import FaultConditionFifteen
from open_fdd.air_handling_unit.reports import FaultCodeFifteenReport

# Create an instance of FaultConditionFifteen
fc15 = FaultConditionFifteen(config_dict)

# Retrieve and print the required columns for this fault condition
fc15_required_columns = fc15.get_required_columns()
print(fc15_required_columns)

# Apply the fault condition to the DataFrame
df_fc15 = fc15.apply(df)

# Calculate and store the fault sum in fault_counts
fault_counts["fc15_fault_sum"] = df_fc15["fc15_flag"].sum()

# Print the fault sum
print(f"FC15 Fault Sum: {fault_counts['fc15_fault_sum']}")

# Retrieve and print all the metrics (keys and values) from summarize_fault_times
fc15_report = FaultCodeFifteenReport(config_dict)
summary_metrics = fc15_report.summarize_fault_times(df_fc15)

# Print each key and value
for key, value in summary_metrics.items():
    print(f"{key}: {value}")

# Generate and display the report if there is a fault
if fault_counts['fc15_fault_sum'] != 0:
    fc15_report.display_report_in_ipython(df_fc15)
else:
    print("No faults found.")


In [ ]:
# Print all fault counts
for fault_name, count in fault_counts.items():
    print(f"{fault_name}: {count}")


In [ ]:
# HEAT MAPS 
# install seaborn with pip

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# Create a copy of df_combined
df_combined_copy = df.copy()

# Resample the data to daily intervals for a clearer heatmap
df_daily = df_combined_copy.resample('D').sum()

# Filter columns ending with '_flag' but exclude those containing 'hour_of_the_day'
fault_columns = [col for col in df_daily.columns if col.endswith('_flag') and 'hour_of_the_day' not in col]


# Subset the DataFrame to include only the fault columns
df_faults = df_daily[fault_columns]

# Transpose for better heatmap visualization
df_faults_t = df_faults.T

# Create a custom color map with more distinction for low values
colors = ["#f0f0f0", "#ffcccc", "#ff6666", "#cc0000", "#660000"]  # light gray to dark red
cmap = LinearSegmentedColormap.from_list("custom_cmap", colors, N=256)

# Plot heatmap using seaborn
plt.figure(figsize=(14, 8))
sns.heatmap(df_faults_t, cmap=cmap, cbar=True, linewidths=0.5)

plt.title('Heatmap of Fault Conditions Over Time')
plt.xlabel('Date')
plt.ylabel('Fault Condition')
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# Create a copy of df_combined
df_fc4_copy = df_fc4.copy()

# Resample the data to daily intervals for a clearer heatmap
df_daily = df_fc4_copy.resample('D').sum()

# Subset the DataFrame to include only the fault columns
df_faults = df_daily[['fc4_flag']]  # Keeping it as a DataFrame with double brackets

# Create a custom color map with more distinction for low values
colors = ["#f0f0f0", "#ffcccc", "#ff6666", "#cc0000", "#660000"]  # light gray to dark red
cmap = LinearSegmentedColormap.from_list("custom_cmap", colors, N=256)

# Plot heatmap using seaborn
plt.figure(figsize=(14, 8))
sns.heatmap(df_faults.T, cmap=cmap, cbar=True, linewidths=0.5)

plt.title('Heatmap of Fault Conditions Over Time')
plt.xlabel('Date')
plt.ylabel('Fault Condition')
plt.show()
